In [ ]:
from curl_cffi import requests as cureq
from rich import print as rprint
from lxml import html
import time

In [7]:
header = {
    #"cookie":"products=; CRAFT_CSRF_TOKEN=77532ba957a0d48a4c79fcf8df36b7785e191e76b289b997ca19a1235b5d902fa%3A2%3A%7Bi%3A0%3Bs%3A16%3A%22CRAFT_CSRF_TOKEN%22%3Bi%3A1%3Bs%3A40%3A%222a2fiP2z-uXH0BjCKYd8hQxzVhgtA6UOVmrv9E_I%22%3B%7D",
    "authority": 'leathercraft-furniture.com',
    "sec-ch-ua":'"Chromium";v="130", "Google Chrome";v="130", "Not?A_Brand";v="99"',
    "user-agent":'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36',    
}

In [213]:
#url = 'https://leathercraft-furniture.com/upholstery/recliners-easebacks'
p_url = 'https://leathercraft-furniture.com/upholstery/chairs'
p2_url = 'https://leathercraft-furniture.com/upholstery/chairs/p2'
willem = 'https://leathercraft-furniture.com/upholstery/139-willem-dining-chair-qs-frame'#139 Willem Dining Chair Product
soho = 'https://leathercraft-furniture.com/upholstery/1652-soho-chair-qs-frame'
collins = 'https://leathercraft-furniture.com/upholstery/1143-3-collins-ottoman'
response = cureq.get(url=willem, impersonate='chrome', headers=header)

In [ ]:
tree = html.fromstring(response.content)
items = tree.xpath('/html/body/div/section/a')
categories = tree.xpath('//*[@id="navmain"]/li[1]/ul/li')
for category in categories[1:]:
    type_url = category.xpath('./a/@href')
    type_name = category.xpath('./a/text()')[0]
    subtypes= category.xpath('./ul/li')
    if subtypes:
        for subtype in subtypes[1:]:
            subtype_url = subtype.xpath('./a/@href')[0]
            subtype_name = subtype.xpath('./a/text()')[0]
            print(f'Type: {type_name} Type URL: {type_url} SubType: {subtype_name} SubType URL: {subtype_url} ')
            #response = cureq.get(url=type_url, impersonate='chrome', headers=header)
            #info = {'Type': type_name, 'Sub-Type': subtype_name}
            #extract(response, info)

    else:
        #response = cureq.get(url=type_url, impersonate='chrome', headers=header)
        #info = {'Type': type_name, 'Sub-Type': None}
        #extract(response, info)
        print(f'Type: {type_name} Type URL: {type_url} || SubType: None')
        continue

            

In [ ]:
tree = html.fromstring(response.content)
next_page = tree.xpath('/html/body/div/div/nav/a[contains(text(), "Next Page")]/@href')[0]
products = tree.xpath('/html/body/div/section/a')

if not next_page:
    for product in products:
        product_url = product.xpath('./@href')[0]
        product_name = product.xpath('./figure/figcaption/text()')
        print(f'Product Name: {product_name[0]} Product URL: {product_url}')
        #response = cureq.get(url=product_url, impersonate='chrome', headers=header)
        #transform(response, info)
        time.sleep(1)
    print(f'Next Page: None')
else:
    for product in products:
        product_url = product.xpath('./@href')[0]
        product_name = product.xpath('./figure/figcaption/text()')
        #response = cureq.get(url=product_url, impersonate='chrome', headers=header)
        #transform(response, info)
        time.sleep(1)
        print(f'Product Name: {product_name[0]} Product URL: {product_url}')
    print(f'Next Page: {next_page}')
    #next_page(next_page)
    

In [211]:
tree = html.fromstring(response.content)

#Extract Product SKU
title = tree.xpath('/html/body/div/article/div[1]/h2/text()')
if title:
    sku = title[0].split(' ')[0]
else:
    sku = None

#Extract Dimensions
dimensions = tree.xpath('/html/body/div/article/div[1]/div[3]/p')[0].text_content()
dim_values = dimensions.replace('\n\t\t\t\t\t\t\t', '%').replace(' \n\t\t\t\t\t\t\t', '%').replace('\t\t\t\t\t\t\t\t\t\t\t\t\t\t', '%')\
    .replace('\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t', '%').replace(' \t\t\t\t\t\t\t\t\t\t\t\t\t', '%')#.replace(' ', '')

all_dimensions= {}
dim_items = dim_values.split('%')
for item in dim_items[1:-1]:
    if item != '':
        item_data = item.split(':')
        all_dimensions[item_data[0]] = item_data[1]


#Extract Pricing Information
fields = [
    "Yardage", "CTN WT", "STD Finish", "Exposed Wood", 
    "STD Trim", "STD Seat Cushion", "STD Back Cushion"
]
pricing_data = {}
for field in fields:
    xpath_expr = f'//div[contains(@class, "product-pricing")]/p/strong[contains(text(), "{field}:")]/following-sibling::text()'
    value = tree.xpath(xpath_expr)
    if value:
        pricing_data[field] = value[0].strip()
    else:
        pricing_data[field] = None


all_data = {
    #**info,
    'Product URL': response.url,
    'Product SKU': sku,
    'Product Images': tree.xpath('/html/body/div/article/div[1]/figure/img/@src'),
    'Product Description': tree.xpath('/html/body/div/article/div[1]/div[2]/p/text()'),
    'Product Dimesions':all_dimensions,
    'Suite': tree.xpath('/html/body/div/article/div[1]/nav[2]/ul/li/a/text()'),
    'Pricing Information': pricing_data,
}

In [212]:
rprint(all_data)

{
    'Product URL': 'https://leathercraft-furniture.com/upholstery/139-willem-dining-chair-qs-frame',
    'Product SKU': '139',
    'Product Images': ['https://leathercraft-furniture.com/assets/old-product-images/139lg.jpg'],
    'Product Description': [
        'Shown with OPTIONAL diamond quadrants on the In Back and Split Panel on the Outback, with #9 Nickel Nail 
trim and Optional "Decorative" handle.'
    ],
    'Product Dimesions': {
        'Outside': ' H 37″ W 19″ D 24″ ',
        'Inside': ' H 16″ W 19″ D 19″ ',
        'Seat Height': ' 21″'
    },
    'Suite': [],
    'Pricing Information': {
        'Yardage': '2',
        'CTN WT': '40',
        'STD Finish': 'Must Specify',
        'Exposed Wood': 'Maple',
        'STD Trim': '#9 Nailhead',
        'STD Seat Cushion': 'Poly Fiber',
        'STD Back Cushion': 'Poly Fiber'
    }
}

In [218]:
tree = html.fromstring(response.content)
dimensions = tree.xpath('/html/body/div/article/div[1]/div[3]/p')[0].text_content()
dim_values = dimensions.replace('\n\t\t\t\t\t\t\t', '%').replace(' \n\t\t\t\t\t\t\t', '%').replace('\t\t\t\t\t\t\t\t\t\t\t\t\t\t', '%')\
    .replace('\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t', '%').replace(' \t\t\t\t\t\t\t\t\t\t\t\t\t', '%')#.replace(' ', '')
#dimensions
all_dimensions= {}
dim_items = dim_values.split('%')
for item in dim_items[1:-1]:
    if item != '':
        item_data = item.split(':')
        all_dimensions[item_data[0]] = item_data[1]

all_dimensions

{'Outside': ' H 37″ W 19″ D 24″ ',
 'Inside': ' H 16″ W 19″ D 19″ ',
 'Seat Height': ' 21″'}